Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.


* Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.
```python
# imaginemos que el valor de una celda es el siguiente
​
'White shark, 3.5 m'
​
# tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
# Un patrón que podriamos usar es: 
​
patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 
​
# de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 
```



* Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.


In [1]:
import pandas as pd
import numpy as np
import sidetable
import re

In [4]:
df = pd.read_csv('attacks_limpieza_1.csv', index_col = 0)
df.head(1)

,case_number,year,type,country,area,location,activity,name,age,time,species_,href,date,month,fatal_y/n,sex_m_f
0,2018.06.25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...,25-Jun-2018,Jun,N,F


In [15]:
# Vamos a sacar los patrones regex da cada especie.
# Para ello, eliminamos temporalmente los nulos para poder filtrar las apariciones de cada tiburón
# Y así sacar el patrón lo más preciso posible

col_species = df['species_'].dropna().reset_index()
col_species

,index,species_
0,0,White shark
1,1,Tiger shark
2,2,"Bull shark, 6'"
3,3,Grey reef shark
4,4,Invalid incident
...,...,...
1541,1646,12' tiger shark
1542,1663,Blue pointer
1543,1664,Blue pointer
1544,1665,Blue pointers


In [18]:
col_species[col_species['species_'].str.contains('tiger', case=False)]['species_'].unique()

array(['Tiger shark', "Tiger shark, 12'", 'Tiger shark, 8 to 10 feet',
       "Tiger shark, 10'", "Tiger shark, 10' ", 'Tiger shark, 2.5 m',
       "Tiger shark, 14'", "Tiger shark, 7'", "Tiger shark, 13' ",
       "Tiger shark, 13'", 'Sandtiger shark', 'Tiger shark, 8 to 12 feet',
       'Tiger shark, 3.5 m', "Tiger shark, 10' to 12'",
       'Tiger shark, 3m ', 'Tiger shark?', "Tiger shark, 8' to 10' ",
       'Tiger shark, 4.8 m', "Tiger shark, 15'", "Tiger shark, 6'",
       "Tiger shark, 8'", "Tiger shark, 10' to 12' ",
       "Tiger shark, 10'  ", 'Tiger shark, 3m',
       'Allegedly a 4 m tiger shark', 'Tiger shark, 2m',
       "Possibly a 5' to 6' sandtiger shark", ' Tiger shark, 2.8m',
       "Sandtiger shark, 4' to 5'", "Sandtiger shark, 8'",
       "Tiger shark, 14' ", "6' to 8' shark, possibly a tiger shark",
       "Tiger shark, 8' to 12'", "Tiger shark, 5' ",
       "Tiger shark, less than 10'", "Tiger shark, 3 m [10']",
       "Sandtiger shark, 2.1 m [7'] ",
       "Tige

In [ ]:
def clasif_especies (fila):
    
    patron_blanco = r".*[Ww](hite|HITE).*"
    if re.findall(patron_blanco, fila):
        fila.replace(fila, 'white shark')
    elif 



2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
* Edad en formato string
* Edades separadas por &, or, to, >
* Edades con ?

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
* Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.


* Puede que os salga un error similar a este:
>```python
>TypeError: expected string or bytes-like object
>```
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
>```python
>df['nombre_columna'] = df['nombre_columna'].astype(str)
>```


* Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método `explode`.



* Por último cambiad el tipo de la columna de string a integer.



3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.
